In [1]:
!pip install numpy==1.19.5 --user

In [2]:
!pip install h5py==2.10.0 --user

In [2]:
!pip install gensim==3.6.0 --user

In [3]:
!pip install deepcut --user

  Using cached https://files.pythonhosted.org/packages/ee/ff/48bde5c0f013094d729fe4b0316ba2a24774b3ff1c52d924a8a4cb04078a/six-1.15.0-py2.py3-none-any.whl
ERROR: tensorflow 2.4.1 has requirement h5py~=2.10.0, but you'll have h5py 3.2.1 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement numpy~=1.19.2, but you'll have numpy 1.20.3 which is incompatible.
  Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0


In [4]:
!pip install pythainlp --user

In [3]:
!pip install nltk --user

In [2]:
from pythainlp.corpus import thai_stopwords
import deepcut
from time import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import datetime
import tensorflow
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Lambda
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.callbacks import ModelCheckpoint

C:\Users\NATTHAWATTUNGRUETHAI\AppData\Roaming\Python\Python38\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [64]:
from flask_ngrok import run_with_ngrok
from flask import Flask, jsonify, request
import json

In [3]:
#Clean Text
def remove_repettition(text):
    token_list = list(text)
    if len(token_list) > 2:
        filter_list = [True, True]
        n = len(token_list)
        for i in range(2, n):
            if (token_list[i] == token_list[i-1]) and (token_list[i] == token_list[i-2]):
                filter_list.append(False)
            else:
                filter_list.append(True)

        output = ''.join(np.array(token_list)[filter_list])
    else:
        output = text
    return output

def cleansing(text):
    # \t, \n, \xa0 and other special characters. Replace by blank string
    text = re.sub('[\t\n\xa0\"\'!?\/\(\)%\:\=\-\+\*\_ๆ]', '', text)
    
    # Numbers. Replace by space
    text = re.sub('[0-9]', ' ', text)
    
    # Dot. Replace by space
    text = re.sub('[\.]', ' ', text)
    
    # One or more consecutive space. Replace by single space
    text = re.sub('\s+',' ',text)
    
    # Remove 2 or more repettition
    text = remove_repettition(text)
    
    return text

In [4]:
import gensim
wv_model = gensim.models.Word2Vec.load('corpus.th.model')

In [5]:
def word2idx(word):
    index = 0
    index = wv_model.wv.vocab[word].index
    return index

In [6]:
def word_index(listword):
    dataset = []
    vocabulary = dict()
    inverse_vocabulary = ['<unk>']  # '<unk>' will never be used, it is only a placeholder for the [0, 0, ....0] embedding
    for sentence in listword:
        tmp = []
        for w in sentence:
            if w not in wv_model:
                continue

            if w not in vocabulary:
                vocabulary[w] = len(inverse_vocabulary)
                tmp.append(len(inverse_vocabulary))
                inverse_vocabulary.append(w)
            else:
                tmp.append(word2idx(w))
        dataset.append(tmp)
    return np.array(dataset)

In [7]:
# define word embedding
vocab_list = [(k, wv_model.wv[k]) for k, v in wv_model.wv.vocab.items()]
embeddings_matrix = np.zeros((len(wv_model.wv.vocab.items()) + 1, wv_model.vector_size))
for i in range(len(vocab_list)):
    word = vocab_list[i][0]
    embeddings_matrix[i + 1] = vocab_list[i][1]

In [8]:
# vocab_list

In [9]:
EMBEDDING_DIM = 300
embeddings_matrix = 1 * np.random.randn(len(vocab_list) + 1, EMBEDDING_DIM)  # This will be the embedding matrix
embeddings_matrix[0] = 0  # So that the padding will be ignored

In [10]:
# Model variables
n_hidden = 256
batch_size = 128
n_epoch = 100
max_seq_length = 2704

In [11]:
embeddings_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.65220199, -0.32731529,  1.22841682, ..., -1.51947888,
        -0.38772034, -0.06970729],
       [-0.66643464,  0.52571597,  1.44998285, ...,  0.26361773,
        -0.19843948, -0.49874863],
       ...,
       [-1.37113419,  1.37313134,  0.9081039 , ..., -0.60839068,
        -0.03872851, -0.48271785],
       [-0.78363534,  1.28484431, -1.04603304, ..., -1.56876537,
        -0.35188567, -1.1400886 ],
       [ 0.46232648,  0.27177069, -0.35248443, ...,  1.50017236,
        -1.67166177,  0.4091093 ]])

In [12]:
def exponent_neg_manhattan_distance(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

In [13]:
# The visible layer
left_input = Input(shape=(max_seq_length,), dtype='int32')
right_input = Input(shape=(max_seq_length,), dtype='int32')

embedding_layer = Embedding(len(embeddings_matrix), EMBEDDING_DIM, weights=[embeddings_matrix], input_length=max_seq_length, trainable=False)

# Embedded version of the inputs
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# Since this is a siamese network, both sides share the same LSTM
shared_lstm = LSTM(n_hidden)

left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)

# Calculates the distance as defined by the MaLSTM model
malstm_distance = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([left_output, right_output])

# Pack it all up into a model
malstm = Model([left_input, right_input], [malstm_distance])


malstm.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

# Start training
training_start_time = time()

In [14]:
malstm.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 2704)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 2704)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 2704, 300)    9468300     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 256)          570368      embedding[0][0]              

In [15]:
# Load best weight from model
malstm.load_weights('sm_colab_ka.h5')

#Test with Text

In [16]:
def prepare_for_predict(input_questions):
    q_input= []
    cleansing(input_questions)
    tokenized_input_1 =deepcut.tokenize(input_questions)
    for sentence in tokenized_input_1:
      q_input.append(sentence)
    q_input= word_index(tokenized_input_1)
    q_input = pad_sequences(q_input, maxlen=max_seq_length)
    return q_input

In [17]:
input1 = 'วิศวคอมเรียนหลักสุตรอะไรบ้างคะ'
input2 = 'วิศวคอมมีหลักสุตรอะไรบ้าง'

In [18]:
input1 = prepare_for_predict(input1)
input2 = prepare_for_predict(input2)

<ipython-input-6-e36182fd4f37>:8: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if w not in wv_model:
<ipython-input-6-e36182fd4f37>:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(dataset)


In [19]:
play_predict = malstm.predict(x=[input1, input2])

In [20]:
print('From model predection of 2 input question pair')
print('Question pairs are not Similar (is_duplicate = 0):\n   {}%'.format(100 - round(play_predict.mean()*100, 2)))
print('Question pairs are Similar (is_duplicate = 1):\n   {}%'.format(round(play_predict.mean()*100, 2)))

From model predection of 2 input question pair
Question pairs are not Similar (is_duplicate = 0):
   39.83%
Question pairs are Similar (is_duplicate = 1):
   60.17%


#Mocking Question from DB

In [21]:
faq_db = pd.read_excel('CPE_FAQ_test.xlsx')
faq_db

,id,qid1,qid2,question1,question2,is_duplicate
0,1,33396,33493,วิศวะคอม/วิศวกรรมคอมพิวเตอร์ มีทั้งหมดกี่หลักส...,วิศวคอมหลักสูตรปกติมีระยะเวลาในการศึกษากี่ปี?,0
1,2,33396,33495,วิศวะคอม/วิศวกรรมคอมพิวเตอร์ มีทั้งหมดกี่หลักส...,วิศวะคอมมีทั้งหมดกี่หลักสูตร? มีหลักสูตรอะไรบ้าง?,1
2,3,33396,33497,วิศวะคอม/วิศวกรรมคอมพิวเตอร์ มีทั้งหมดกี่หลักส...,วิศวกรรมคอมพิวเตอร์มีทั้งหมดกี่หลักสูตร? มีหลั...,1
3,4,33396,33499,วิศวะคอม/วิศวกรรมคอมพิวเตอร์ มีทั้งหมดกี่หลักส...,วิศวะคอมมีทั้งหมดกี่หลักสูตร?,1
4,5,33396,33501,วิศวะคอม/วิศวกรรมคอมพิวเตอร์ มีทั้งหมดกี่หลักส...,วิศวกรรมคอมพิวเตอร์มีทั้งหมดกี่หลักสูตร?,1
...,...,...,...,...,...,...
82,83,33441,33261,วิศวะคอม/วิศวกรรมคอมพิวเตอร์หลักสูตรนานาชาติ ป...,วิศวคอมพิวพิวเตอร์หลักสูตรปกติ ปีที่1-2 เรียนอ...,0
83,84,33443,33251,วิศวะคอม/วิศวกรรมคอมพิวเตอร์หลักสูตรนานาชาติปี...,วิศวคอมพิวพิวเตอร์หลักสูตรปกติ ปีที่ 3 เรียนอะ...,0
84,85,33445,33241,วิศวะคอม/วิศวกรรมคอมพิวเตอร์หลักสูตรนานาชาติ ป...,วิศวคอมพิวพิวเตอร์หลักสูตรปกติ ปีที่ 4 เรียนอะ...,0
85,86,33447,33231,วิศวะคอม/วิศวกรรมคอมพิวเตอร์หลักสูตรนานาชาติ ป...,วิศวคอมพิวพิวเตอร์หลักสูตรนานาชาติ ปีที่1-2 เร...,0


In [22]:
faq_db['question1']

0     วิศวะคอม/วิศวกรรมคอมพิวเตอร์ มีทั้งหมดกี่หลักส...
1     วิศวะคอม/วิศวกรรมคอมพิวเตอร์ มีทั้งหมดกี่หลักส...
2     วิศวะคอม/วิศวกรรมคอมพิวเตอร์ มีทั้งหมดกี่หลักส...
3     วิศวะคอม/วิศวกรรมคอมพิวเตอร์ มีทั้งหมดกี่หลักส...
4     วิศวะคอม/วิศวกรรมคอมพิวเตอร์ มีทั้งหมดกี่หลักส...
                            ...                        
82    วิศวะคอม/วิศวกรรมคอมพิวเตอร์หลักสูตรนานาชาติ ป...
83    วิศวะคอม/วิศวกรรมคอมพิวเตอร์หลักสูตรนานาชาติปี...
84    วิศวะคอม/วิศวกรรมคอมพิวเตอร์หลักสูตรนานาชาติ ป...
85    วิศวะคอม/วิศวกรรมคอมพิวเตอร์หลักสูตรนานาชาติ ป...
86    วิศวะคอม/วิศวกรรมคอมพิวเตอร์หลักสูตรนานาชาติปี...
Name: question1, Length: 87, dtype: object

In [23]:
from itertools import chain
def tokenize_text_list(ls):
    """Tokenize list of text"""
    return list(chain.from_iterable([deepcut.tokenize(ls)]))

In [24]:
tokenized_category =faq_db.question1.map(tokenize_text_list)

In [25]:
tokenized_category

0     [วิศวะ, คอม, /, วิศวกรรม, คอมพิวเตอร์,  , มี, ...
1     [วิศวะ, คอม, /, วิศวกรรม, คอมพิวเตอร์,  , มี, ...
2     [วิศวะ, คอม, /, วิศวกรรม, คอมพิวเตอร์,  , มี, ...
3     [วิศวะ, คอม, /, วิศวกรรม, คอมพิวเตอร์,  , มี, ...
4     [วิศวะ, คอม, /, วิศวกรรม, คอมพิวเตอร์,  , มี, ...
                            ...                        
82    [วิศวะ, คอม, /, วิศวกรรม, คอมพิวเตอร์, หลักสูต...
83    [วิศวะ, คอม, /, วิศวกรรม, คอมพิวเตอร์, หลักสูต...
84    [วิศวะ, คอม, /, วิศวกรรม, คอมพิวเตอร์, หลักสูต...
85    [วิศวะ, คอม, /, วิศวกรรม, คอมพิวเตอร์, หลักสูต...
86    [วิศวะ, คอม, /, วิศวกรรม, คอมพิวเตอร์, หลักสูต...
Name: question1, Length: 87, dtype: object

In [26]:
max_word = 19219
max_seq_length = 2704

In [27]:
q_category= []
for sentence in tokenized_category:
    q_category.append(sentence)

In [28]:
import difflib
q_category = word_index(q_category)

<ipython-input-6-e36182fd4f37>:8: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if w not in wv_model:
<ipython-input-6-e36182fd4f37>:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(dataset)


In [29]:
q_category.shape

(87,)

In [30]:
all_Question_categorylen = len(q_category)
all_Question_categorylen

87

#Question from User

In [31]:
input2 = 'วิศวคอมมีหลักสุตรอะไรบ้าง'

In [32]:
cleansing(input2)
tokenized_input_2 =deepcut.tokenize(input2)

In [33]:
tokenized_input_2

['วิศว', 'คอม', 'มี', 'หลักสุตร', 'อะไร', 'บ้าง']

In [34]:
#Duplicate list
def duplicate(testList, n):
    return [ele for ele in testList for _ in range(n)]

In [35]:
tokenized_dup_input_2= duplicate([tokenized_input_2],all_Question_categorylen)
tokenized_dup_input_2

[['วิศว', 'คอม', 'มี', 'หลักสุตร', 'อะไร', 'บ้าง'],
 ['วิศว', 'คอม', 'มี', 'หลักสุตร', 'อะไร', 'บ้าง'],
 ['วิศว', 'คอม', 'มี', 'หลักสุตร', 'อะไร', 'บ้าง'],
 ['วิศว', 'คอม', 'มี', 'หลักสุตร', 'อะไร', 'บ้าง'],
 ['วิศว', 'คอม', 'มี', 'หลักสุตร', 'อะไร', 'บ้าง'],
 ['วิศว', 'คอม', 'มี', 'หลักสุตร', 'อะไร', 'บ้าง'],
 ['วิศว', 'คอม', 'มี', 'หลักสุตร', 'อะไร', 'บ้าง'],
 ['วิศว', 'คอม', 'มี', 'หลักสุตร', 'อะไร', 'บ้าง'],
 ['วิศว', 'คอม', 'มี', 'หลักสุตร', 'อะไร', 'บ้าง'],
 ['วิศว', 'คอม', 'มี', 'หลักสุตร', 'อะไร', 'บ้าง'],
 ['วิศว', 'คอม', 'มี', 'หลักสุตร', 'อะไร', 'บ้าง'],
 ['วิศว', 'คอม', 'มี', 'หลักสุตร', 'อะไร', 'บ้าง'],
 ['วิศว', 'คอม', 'มี', 'หลักสุตร', 'อะไร', 'บ้าง'],
 ['วิศว', 'คอม', 'มี', 'หลักสุตร', 'อะไร', 'บ้าง'],
 ['วิศว', 'คอม', 'มี', 'หลักสุตร', 'อะไร', 'บ้าง'],
 ['วิศว', 'คอม', 'มี', 'หลักสุตร', 'อะไร', 'บ้าง'],
 ['วิศว', 'คอม', 'มี', 'หลักสุตร', 'อะไร', 'บ้าง'],
 ['วิศว', 'คอม', 'มี', 'หลักสุตร', 'อะไร', 'บ้าง'],
 ['วิศว', 'คอม', 'มี', 'หลักสุตร', 'อะไร', 'บ้าง'],
 ['วิศว', 'ค

In [36]:
q_user = word_index(tokenized_dup_input_2)

<ipython-input-6-e36182fd4f37>:8: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if w not in wv_model:


In [37]:
# Split to dicts
M_input = {'left': q_category, 'right': q_user}

In [38]:
# Zero padding
for model_input, side in itertools.product([M_input], ['left', 'right']):
    model_input[side] = pad_sequences(model_input[side], maxlen=max_seq_length)

# Make sure everything is ok
assert M_input['left'].shape == M_input['right'].shape

In [39]:
M_input['left']

array([[  0,   0,   0, ..., 237,   9,  10],
       [  0,   0,   0, ..., 237,  29,  38],
       [  0,   0,   0, ..., 237,  29,  38],
       ...,
       [  0,   0,   0, ...,  98,  29,  38],
       [  0,   0,   0, ...,  98,  29,  38],
       [  0,   0,   0, ...,  98,  29,  38]])

In [40]:
 M_input['right']

array([[    0,     0,     0, ...,     4,     5,     6],
       [    0,     0,     0, ..., 22343,    29,    38],
       [    0,     0,     0, ..., 22343,    29,    38],
       ...,
       [    0,     0,     0, ..., 22343,    29,    38],
       [    0,     0,     0, ..., 22343,    29,    38],
       [    0,     0,     0, ..., 22343,    29,    38]])

In [41]:
play_predict = malstm.predict(x=[M_input['left'],  M_input['right']])

In [42]:
play_predict

array([[5.1643596e-22],
       [9.7786230e-01],
       [9.7786230e-01],
       [9.7786230e-01],
       [9.7786230e-01],
       [9.7786230e-01],
       [9.7786230e-01],
       [9.7786230e-01],
       [9.7786230e-01],
       [9.7786230e-01],
       [9.7786230e-01],
       [9.7786230e-01],
       [9.7786230e-01],
       [9.7786230e-01],
       [9.7786230e-01],
       [9.7786230e-01],
       [9.7786230e-01],
       [9.7786230e-01],
       [9.7786230e-01],
       [9.7786230e-01],
       [9.7786230e-01],
       [9.7786230e-01],
       [9.7786230e-01],
       [9.7786230e-01],
       [9.7786230e-01],
       [9.7786230e-01],
       [9.7786230e-01],
       [9.7786230e-01],
       [9.7786230e-01],
       [9.7786230e-01],
       [9.7786236e-01],
       [9.7786236e-01],
       [9.7786230e-01],
       [9.7786230e-01],
       [9.7786230e-01],
       [9.7786230e-01],
       [9.7786230e-01],
       [9.7786230e-01],
       [9.7786230e-01],
       [9.2381722e-01],
       [9.0207475e-01],
       [9.020747

In [57]:
max(play_predict)

array([0.97786236], dtype=float32)

In [58]:
max_question_percentage = max(play_predict)

In [60]:
print(max_question_percentage[0])

0.97786236


In [61]:
question_index = np.where(play_predict == max_question_percentage)

In [62]:
question_index[0][0]

30

In [47]:
faq_db['question1']

0     วิศวะคอม/วิศวกรรมคอมพิวเตอร์ มีทั้งหมดกี่หลักส...
1     วิศวะคอม/วิศวกรรมคอมพิวเตอร์ มีทั้งหมดกี่หลักส...
2     วิศวะคอม/วิศวกรรมคอมพิวเตอร์ มีทั้งหมดกี่หลักส...
3     วิศวะคอม/วิศวกรรมคอมพิวเตอร์ มีทั้งหมดกี่หลักส...
4     วิศวะคอม/วิศวกรรมคอมพิวเตอร์ มีทั้งหมดกี่หลักส...
                            ...                        
82    วิศวะคอม/วิศวกรรมคอมพิวเตอร์หลักสูตรนานาชาติ ป...
83    วิศวะคอม/วิศวกรรมคอมพิวเตอร์หลักสูตรนานาชาติปี...
84    วิศวะคอม/วิศวกรรมคอมพิวเตอร์หลักสูตรนานาชาติ ป...
85    วิศวะคอม/วิศวกรรมคอมพิวเตอร์หลักสูตรนานาชาติ ป...
86    วิศวะคอม/วิศวกรรมคอมพิวเตอร์หลักสูตรนานาชาติปี...
Name: question1, Length: 87, dtype: object

In [48]:
faq_db.loc[30,'question2']

'วิศวะคอม/วิศวกรรมคอมพิวเตอร์หลักสูตรปกติ  ปี 4 เทอม 2\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?'

In [49]:
faq_db.loc[30,'question1']

'วิศวะคอม/วิศวกรรมคอมพิวเตอร์ มีทั้งหมดกี่หลักสูตร? มีหลักสูตรอะไรบ้าง?'

In [71]:
value = {
          "predictedQuestion": faq_db.loc[30,'question1'],
          "similarity": json.dumps(str(max_question_percentage[0]))
        }
json.dumps(value, ensure_ascii=False).encode('utf8')

b'{"predictedQuestion": "\xe0\xb8\xa7\xe0\xb8\xb4\xe0\xb8\xa8\xe0\xb8\xa7\xe0\xb8\xb0\xe0\xb8\x84\xe0\xb8\xad\xe0\xb8\xa1/\xe0\xb8\xa7\xe0\xb8\xb4\xe0\xb8\xa8\xe0\xb8\xa7\xe0\xb8\x81\xe0\xb8\xa3\xe0\xb8\xa3\xe0\xb8\xa1\xe0\xb8\x84\xe0\xb8\xad\xe0\xb8\xa1\xe0\xb8\x9e\xe0\xb8\xb4\xe0\xb8\xa7\xe0\xb9\x80\xe0\xb8\x95\xe0\xb8\xad\xe0\xb8\xa3\xe0\xb9\x8c \xe0\xb8\xa1\xe0\xb8\xb5\xe0\xb8\x97\xe0\xb8\xb1\xe0\xb9\x89\xe0\xb8\x87\xe0\xb8\xab\xe0\xb8\xa1\xe0\xb8\x94\xe0\xb8\x81\xe0\xb8\xb5\xe0\xb9\x88\xe0\xb8\xab\xe0\xb8\xa5\xe0\xb8\xb1\xe0\xb8\x81\xe0\xb8\xaa\xe0\xb8\xb9\xe0\xb8\x95\xe0\xb8\xa3? \xe0\xb8\xa1\xe0\xb8\xb5\xe0\xb8\xab\xe0\xb8\xa5\xe0\xb8\xb1\xe0\xb8\x81\xe0\xb8\xaa\xe0\xb8\xb9\xe0\xb8\x95\xe0\xb8\xa3\xe0\xb8\xad\xe0\xb8\xb0\xe0\xb9\x84\xe0\xb8\xa3\xe0\xb8\x9a\xe0\xb9\x89\xe0\xb8\xb2\xe0\xb8\x87?", "similarity": "0.97786236"}'

In [74]:
json.dumps(str(max_question_percentage[0]))

'"0.97786236"'